In [24]:
import numpy as np
import pandas as pd
from suffix_trees.STree import STree

In [153]:
# Загружаем подготовленные датасеты с городскими и коммерческими маршрутами 
# (с полными списками остановок в обоих направлениях и номерами кластеров, к которым относятся остановки)
r_df = pd.read_csv('all_routes_info.csv', index_col=0)
all_routes_df = r_df[['route_number', 'id_transport', 'number_of_stops_in_direction1', 'stops_in_direction1', 'number_of_stops_in_direction2', 'stops_in_direction2', 'clusters_in_direction1', 'clusters_in_direction2']]

comm_df = pd.read_csv('commerce_stops.csv', index_col=0)
comm_df[comm_df['route_number'] == 191]['clusters_in_direction1'].values[0]

'2058,2210,2338,1940,1752,2529,1712,1716,603,600,606,1216,1740,1821,1743,1741,203,203,2185,1491,198,878,361,877,875,876,876,1018,1498,1020,2179,1455,860,294,1364,2325,416'

In [117]:
# Основной алгоритм поиска похожих городских маршрутов. Состоит из нескольких функций

#def find_longest_subroute(str1, str2):

def sequential_slice(iterable, length):
    pool = tuple(iterable)
    assert 0 < length <= len(pool)
    tails = (pool[s:] for s in range(length))
    return zip(*tails)

def sequence_in_list(sequence, lst):
    pool = tuple(sequence)
    return any((pool == s for s in sequential_slice(lst, len(pool))))

def lcs(str1, str2):
    a = str1.split(',')
    b = str2.split(',')
    
    is_max = True
    if len(a) > len(b):
        a, b = b, a
    for l in reversed(range(1, len(a)+1)):
        seq = [subseq for subseq in sequential_slice(a, l) if sequence_in_list(subseq, b)]
        if seq:
            break
    return seq

def create_list_of_subroutes(str1, str2):
    res = 1
    all_subroutes = []
    while len(str1) > 0 and len(str2) > 0 and res != 0:
        seq = lcs(str1, str2)
        if len(seq) == 0:
            res = 0
        else:
            all_subroutes.append(seq[0])
            substr = ""
            for i in seq[0]:
                substr += "," + i
            
            new_str1_array = str1.split(substr)
            if len(new_str1_array) == 1:
                str1 = str1.split(substr)[0]
            else:
                str1 = str1.split(substr)[0] + str1.split(substr)[1]
                
            new_str2_array = str2.split(substr) 
            if len(new_str2_array) == 1:
                str2 = str2.split(substr)[0]
            else:
                str2 = str2.split(substr)[0] + str2.split(substr)[1]
    amount_of_similar = 0            
    for i in range(len(all_subroutes)):
        amount_of_similar += len(all_subroutes[i]) - 1
    return all_subroutes, amount_of_similar  

In [118]:
all_routes_names = all_routes_df['route_number'].values

In [163]:
def find_similar_routes(comm_route):
    comm_clusters1 = comm_df[comm_df['route_number'] == comm_route]['clusters_in_direction1'].values[0]
    comm_clusters2 = comm_df[comm_df['route_number'] == comm_route]['clusters_in_direction2'].values[0]
    
    number_of_stops = len(comm_clusters1.split(','))
    choosen_direction = 1
    direction_of_result = 1
    
    similarity_arr = []
    max_subroute = []
    if comm_clusters1 != '-1':
        for route in all_routes_names:
            current_clusters = all_routes_df[all_routes_df['route_number'] == route]['clusters_in_direction1'].values[0]
            subroutes, amount = create_list_of_subroutes(comm_clusters1, current_clusters)
            if len(subroutes) == 0:
                similarity_arr.append([route, 0, []])
            else:                       
                similarity_arr.append([route, amount, subroutes])
    most_similar = sorted(similarity_arr, key=lambda x: x[1], reverse=True)[:5]
    return most_similar, number_of_stops, choosen_direction, direction_of_result

def perform_result(comm_route):
    similar, number_of_stops,  choosen_direction, direction_of_result = find_similar_routes(comm_route)
    print('-------------------------------------')
    print('Для коммерческого маршрута № '+ str(comm_route) + ' найдено пять наиболее похожих:')
    for route in similar:
        print("Маршрут № " + route[0] + ", процент совпадения: ", round(route[1] * 100 / (number_of_stops - 1)), "%")
        print("На данном маршруте совпадают следующие перегоны (пока в виде id кластеров): ", route[2])
        print()

In [164]:
route1 = "15,17,18,19,20,33,45,56"
route2 = "9,6,3,9,18,19,20,34,45,56"
x1 = create_list_of_subroutes(route1, route2)
#x1= lcs(route1, route2)
x1[1]

3

In [165]:
perform_result(191)

-------------------------------------
Для коммерческого маршрута № 191 найдено пять наиболее похожих:
Маршрут № 103, процент совпадения:  39 %
На данном маршруте совпадают следующие перегоны (пока в виде id кластеров):  [('1821', '1743', '1741', '203', '203', '2185'), ('1716', '603', '600', '606', '1216'), ('1491', '198', '878', '361'), ('1940', '1752', '2529'), ('2058',)]

Маршрут № 107, процент совпадения:  36 %
На данном маршруте совпадают следующие перегоны (пока в виде id кластеров):  [('876', '1018', '1498', '1020', '2179', '1455', '860', '294', '1364', '2325'), ('198', '878', '361', '877', '875')]

Маршрут № 102, процент совпадения:  36 %
На данном маршруте совпадают следующие перегоны (пока в виде id кластеров):  [('1821', '1743', '1741', '203', '203', '2185'), ('603', '600', '606', '1216'), ('1491', '198', '878', '361'), ('1940', '1752', '2529'), ('2058',)]

Маршрут № 153, процент совпадения:  33 %
На данном маршруте совпадают следующие перегоны (пока в виде id кластеров):  [(

In [34]:
x = route1.split(',')
y = route2.split(',')
LCS_DYN(x, y)

['18', '19', '20', '56', '68', '67']